<a href="https://colab.research.google.com/github/JimMcDonough/Capstone/blob/main/distillbert_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 11 18:02:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.4 MB/s 
     |████████████████████████████████| 7.6 MB 53.0 MB/s 
     |████████████████████████████████| 163 kB 54.4 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 27.8 MB/s 
     |████████████████████████████████| 95 kB 5.8 MB/s 
     |████████████████████████████████| 115 kB 76.3 MB/s 
     |████████████████████████████████| 212 kB 68.0 MB/s 
     |████████████████████████████████| 127 kB 54.3 MB/s 
     |████████████████████████████████| 115 kB 57.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"/content/drive/My Drive/sample_data/train.json",
           "validation": r"/content/drive/My Drive/sample_data/val.json",
           "test": r"/content/drive/My Drive/sample_data/test.json"}

In [ ]:
features = Features({'answers': [{'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}],
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [ ]:
dataset = load_dataset("json", data_files=data_files, features=features, cache_dir= r"/content/drive/My Drive/sample_data/")


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",     #only truncates the context nothing else
        return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    #print(offset_mapping)
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    #print(answers)
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
        sequence_ids = inputs.sequence_ids(i)  #no clue what this does
        #print(answer, end_char, sequence_ids)
        # Find the start and end of the context        ; not sure how this section works
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
    
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

In [ ]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
batch_sizes = [8,16,24]
epoch_sizes = [3,4]
learning_rates = [0.00032, 0.000192, 0.000128, 0.000064]

In [ ]:
results_list = []
for num_epochs in epoch_sizes:
    for batch_size in batch_sizes:
        for learning_rate in learning_rates:
            result = {'epochs':num_epochs,
                     'batch_size': batch_size,
                     'learning_rate':learning_rate}
            
            model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
            
            tf_train_set = model.prepare_tf_dataset(
                tokenized_squad["train"],
                shuffle=True,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            tf_validation_set = model.prepare_tf_dataset(
                tokenized_squad["validation"],
                shuffle=False,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            num_train_steps = len(tf_train_set) * num_epochs
            lr_scheduler = PolynomialDecay(
                initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
            )
            
            opt = Adam(learning_rate=lr_scheduler)
            
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
            
            history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)
            result['val_end_accuracy'] = history.history['val_end_logits_accuracy']
            result['val_start_accuracy'] = history.history['val_start_logits_accuracy']
            results_list.append(result) 
            del history
            del model
            del tf_train_set
            del tf_validation_set

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1575/1575 [==============================] - 624s 392ms/step - loss: 6.8375 - end_logits_loss: 3.3542 - start_logits_loss: 3.4832 - end_logits_accuracy: 0.1877 - start_logits_accuracy: 0.1846 - val_loss: 5.7095 - val_end_logits_loss: 2.7716 - val_start_logits_loss: 2.9379 - val_end_logits_accuracy: 0.3149 - val_start_logits_accuracy: 0.2904
Epoch 2/3
1575/1575 [==============================] - 623s 396ms/step - loss: 4.2684 - end_logits_loss: 2.0698 - start_logits_loss: 2.1985 - end_logits_accuracy: 0.4380 - start_logits_accuracy: 0.4213 - val_loss: 5.2130 - val_end_logits_loss: 2.5002 - val_start_logits_loss: 2.7128 - val_end_logits_accuracy: 0.3870 - val_start_logits_accuracy: 0.3489
Epoch 3/3
1575/1575 [==============================] - 622s 395ms/step - loss: 1.9354 - end_logits_loss: 0.9169 - start_logits_loss: 1.0185 - end_logits_accuracy: 0.7249 - start_logits_accuracy: 0.6987 - val_loss: 5.8526 - val_end_logits_loss: 2.8510 - val_start_logits_loss: 3.0016 - val_end_l

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1575/1575 [==============================] - 630s 397ms/step - loss: 4.8459 - end_logits_loss: 2.3581 - start_logits_loss: 2.4878 - end_logits_accuracy: 0.3955 - start_logits_accuracy: 0.3789 - val_loss: 3.6411 - val_end_logits_loss: 1.7339 - val_start_logits_loss: 1.9072 - val_end_logits_accuracy: 0.5603 - val_start_logits_accuracy: 0.4972
Epoch 2/3
1575/1575 [==============================] - 623s 395ms/step - loss: 2.4341 - end_logits_loss: 1.1686 - start_logits_loss: 1.2655 - end_logits_accuracy: 0.6705 - start_logits_accuracy: 0.6363 - val_loss: 3.5032 - val_end_logits_loss: 1.6641 - val_start_logits_loss: 1.8390 - val_end_logits_accuracy: 0.5717 - val_start_logits_accuracy: 0.5326
Epoch 3/3
1575/1575 [==============================] - 623s 395ms/step - loss: 0.9889 - end_logits_loss: 0.4753 - start_logits_loss: 0.5136 - end_logits_accuracy: 0.8497 - start_logits_accuracy: 0.8329 - val_loss: 4.2282 - val_end_logits_loss: 1.9769 - val_start_logits_loss: 2.2513 - val_end_l

...

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs', 'dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
787/787 [==============================] - 604s 761ms/step - loss: 4.0605 - end_logits_loss: 1.9863 - start_logits_loss: 2.0742 - end_logits_accuracy: 0.4894 - start_logits_accuracy: 0.4607 - val_loss: 2.9558 - val_end_logits_loss: 1.4046 - val_start_logits_loss: 1.5512 - val_end_logits_accuracy: 0.6221 - val_start_logits_accuracy: 0.5828
Epoch 2/3
787/787 [==============================] - 598s 759ms/step - loss: 1.8606 - end_logits_loss: 0.9081 - start_logits_loss: 0.9525 - end_logits_accuracy: 0.7335 - start_logits_accuracy: 0.7109 - val_loss: 3.0675 - val_end_logits_loss: 1.4435 - val_start_logits_loss: 1.6240 - val_end_logits_accuracy: 0.6383 - val_start_logits_accuracy: 0.5855
Epoch 3/3
787/787 [==============================] - 598s 759ms/step - loss: 0.9823 - end_logits_loss: 0.4827 - start_logits_loss: 0.4996 - end_logits_accuracy: 0.8499 - start_logits_accuracy: 0.8374 - val_loss: 3.5283 - val_end_logits_loss: 1.6466 - val_start_logits_loss: 1.8816 - val_end_logits_

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs', 'dropout_179']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


ResourceExhaustedError: ignored

# New Section

# New Section

In [ ]:
results_list[1]['history'].history

{'loss': [4.845857620239258, 2.4341084957122803, 0.9889172315597534],
 'end_logits_loss': [2.3580968379974365,
  1.1686468124389648,
  0.47533130645751953],
 'start_logits_loss': [2.487758159637451,
  1.2654602527618408,
  0.5135844349861145],
 'end_logits_accuracy': [0.39547619223594666,
  0.6704761981964111,
  0.8496825098991394],
 'start_logits_accuracy': [0.378888875246048,
  0.6363492012023926,
  0.8329365253448486],
 'val_loss': [3.64108943939209, 3.5031754970550537, 4.228151798248291],
 'val_end_logits_loss': [1.7338542938232422,
  1.6641385555267334,
  1.9768900871276855],
 'val_start_logits_loss': [1.9072333574295044,
  1.8390371799468994,
  2.2512638568878174],
 'val_end_logits_accuracy': [0.560252845287323,
  0.5717107653617859,
  0.5883049964904785],
 'val_start_logits_accuracy': [0.4972342848777771,
  0.5325958132743835,
  0.5466218590736389]}

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
1575/1575 [==============================] - 634s 398ms/step - loss: 3.8666 - end_logits_loss: 1.8963 - start_logits_loss: 1.9704 - end_logits_accuracy: 0.5010 - start_logits_accuracy: 0.4844 - val_loss: 3.0543 - val_end_logits_loss: 1.4584 - val_start_logits_loss: 1.5959 - val_end_logits_accuracy: 0.6092 - val_start_logits_accuracy: 0.5739
Epoch 2/3
1575/1575 [==============================] - 628s 399ms/step - loss: 1.7818 - end_logits_loss: 0.8772 - start_logits_loss: 0.9046 - end_logits_accuracy: 0.7390 - start_logits_accuracy: 0.7247 - val_loss: 3.0394 - val_end_logits_loss: 1.4328 - val_start_logits_loss: 1.6066 - val_end_logits_accuracy: 0.6328 - val_start_logits_accuracy: 0.5928
Epoch 3/3
1575/1575 [==============================] - 628s 398ms/step - loss: 0.8947 - end_logits_loss: 0.4467 - start_logits_loss: 0.4480 - end_logits_accuracy: 0.8596 - start_logits_accuracy: 0.8525 - val_loss: 3.5791 - val_end_logits_loss: 1.6970 - val_start_logits_loss: 1.8821 - val_end_logits_accuracy: 0.6333 - val_start_logits_accuracy: 0.5927
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
 585/1575 [==========>...................] - ETA: 5:48 - loss: 5.0207 - end_logits_loss: 2.4658 - start_logits_loss: 2.5549 - end_logits_accuracy: 0.3863 - start_logits_accuracy: 0.3744

In [ ]:
#del history
del model
del tf_train_set
del tf_validation_set

In [ ]:
batch_sizes = [24]
epoch_sizes = [3]
learning_rates = [0.00032, 0.000192, 0.000128, 0.000064]

In [ ]:
results_list = []
for num_epochs in epoch_sizes:
    for batch_size in batch_sizes:
        for learning_rate in learning_rates:
            result = {'epochs':num_epochs,
                     'batch_size': batch_size,
                     'learning_rate':learning_rate}
            
            model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
            
            tf_train_set = model.prepare_tf_dataset(
                tokenized_squad["train"],
                shuffle=True,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            tf_validation_set = model.prepare_tf_dataset(
                tokenized_squad["validation"],
                shuffle=False,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            num_train_steps = len(tf_train_set) * num_epochs
            lr_scheduler = PolynomialDecay(
                initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
            )
            
            opt = Adam(learning_rate=lr_scheduler)
            
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
            
            history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)
            result['val_end_accuracy'] = history.history['val_end_logits_accuracy']
            result['val_start_accuracy'] = history.history['val_start_logits_accuracy']
            results_list.append(result) 
            del history
            del model
            del tf_train_set
            del tf_validation_set

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs', 'dropout_199']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


ResourceExhaustedError: ignored